In [1]:
# Understanding Decorators
# https://realpython.com/primer-on-python-decorators/

In [2]:
## Step 1
## Python's Functions are First-Class Objects
## https://dbader.org/blog/python-first-class-functions 

In [8]:
## Functions are objects
def yell(text):
    return text.upper() + '!'
bark = yell
del yell
bark.__name__

'yell'

In [11]:
## Functions can be stored in data structures
funcs = [bark, str.lower]
print(funcs)
funcs[0]('magic')

[<function yell at 0x7f4590ce7ea0>, <method 'lower' of 'str' objects>]


'MAGIC!'

In [17]:
## Functions can be passed to other functions
def greet(func):
    greeting = func('Hi, I am a Python program')
    print(greeting)

def whisper(text):
    return text.lower() + '...'
    
greet(bark)
greet(whisper)

## Functions which can accept other functions as arguments are called higher-order functions
list(map(bark, ['hello', 'hey', 'hi']))

HI, I AM A PYTHON PROGRAM!
hi, i am a python program...


['HELLO!', 'HEY!', 'HI!']

In [22]:
## functions can be nested
def speak(text):
    def iwhisper(t):
        return t.lower() + '...'
    print(iwhisper(text))
speak('Hello, World')

# iwhisper('YO') # local function
# speak.iwhisper # ^^

hello, world...


In [28]:
def get_speak_func(volume):
    def whisper(text):
        return text.lower() + '...'
    def yell(text):
        return text.upper() + '!'
    if volume > 0.5:
        return yell
    else:
        return whisper
print(get_speak_func(0.3))
print(get_speak_func(0.7))
print(get_speak_func(0.3)('Hello'))
print(get_speak_func(0.7)('Hello'))

<function get_speak_func.<locals>.whisper at 0x7f457bf50f28>
<function get_speak_func.<locals>.yell at 0x7f457b13dae8>
hello...
HELLO!


In [33]:
## functions can capture local state
def get_speak_func(text, volume):
    def whisper():
        return text.lower() + '...'
    def yell():
        return text.upper() + '!'
    if volume > 0.5:
        return yell
    else:
        return whisper
print(get_speak_func('Hello, World', 0.7)())

###  A function like this (closure) remembers the values from its enclosing lexical scope 
### even when the program flow is no longer in that scope.

def make_adder(n):
    def add(x):
        return x + n
    return add
print(make_adder(3)(5))
print(make_adder(5)(5))

HELLO, WORLD!
8
10


In [35]:
## Objects can behave like functions
class Adder:
    def __init__(self, n):
         self.n = n
    def __call__(self, x):
        return self.n + x
Adder(3)(4)
Adder(6)(8)

14

In [36]:
## Step 2
## Decorators

In [38]:
# Simple Wrapping
def my_decorator(func):
    def wrapper():
        print("Before the function is called")
        func()
        print("After the function is called")
    return wrapper
def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee)
say_whee()

Before the function is called
Whee!
After the function is called


In [44]:
#Wrapping with parameters
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

def say_whee():
    print("Whee!")

say_whee = not_during_the_night(say_whee)
say_whee()

Whee!


In [45]:
# Using Decorators' syntax
def my_decorator(func):
    def wrapper():
        print("Before")
        func()
        print("After")
    return wrapper

@my_decorator
def say_whee():
    print("Whee!")
    
say_whee()

Before
Whee!
After


In [46]:
# Reusing decorators

def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

@do_twice
def say_whee():
    print("Whee!")
    
say_whee()

Whee!
Whee!


In [48]:
## Decorating Functions with Arguments

def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")
    
greet("Vader!")

Hello Vader!
Hello Vader!


In [52]:
## Returning Values from Decorated Functions

@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"
# hi_adam = return_greeting("Adam")
# print(hi_adam)

def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

print(hi_adam)

Hi Adam


In [53]:
## introspection
## Introspection is the ability of an object to know about its own attributes at runtime.
print(say_whee)
print(say_whee.__name__)
print(help(say_whee))

<function do_twice.<locals>.wrapper_do_twice at 0x7f457bb6c0d0>
wrapper_do_twice
Help on function wrapper_do_twice in module __main__:

wrapper_do_twice()

None


In [57]:
## If we want decorated functions to retain their identity, we should use @functools.wraps decorator
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def say_whee():
    print("Whee!")

print(say_whee)
print(say_whee.__name__)
print(help(say_whee))

<function say_whee at 0x7f457bb6c488>
say_whee
Help on function say_whee in module __main__:

say_whee()

None
